In [254]:
from echo_api import Connection, Settings, APICallError
from xmlmanip import XMLSchema
import xml.etree.ElementTree as ET
from requests import Session
from functools import wraps
import uuid

from zeep import Client
from zeep.transports import Transport


'1c71baf24edd43918cd7e40c0b38564d'

In [3]:
def handle_response(method):
    @wraps(method)
    def _impl(self, *method_args, **method_kwargs):
        response = method(self, *method_args, **method_kwargs)
        if "Error|" in response:
            raise APICallError(response)
        else:
            return response
    return _impl

In [368]:
class Connection:
    def _get_physician_guid(self, physician_id):
        qs = f"SELECT * FROM PhysicianDetail WHERE PhysicianID={physician_id}"
        schema_str = self.client.service.API_GeneralQuery(self.session_id, qs, "")
        schema = XMLSchema(schema_str)
        paths = schema.locate(PhysicianID__ne='-1')
        physicians = [schema.retrieve('__'.join(path.split('__')[:-1])) for path in paths]
        if len(physicians) > 0:
            physician = sorted(physicians, key=lambda x: int(x['PhysicianID']), reverse=True)[0]
        else:
            raise APICallError(f'No Physicians matching id {physician_id}')
        return physician["EntityGuid"]
    
    def get_latest_contact_log(self, physician_id):
        qs = "SELECT * FROM ContactLog"
        schema_str = self.client.service.API_GeneralQuery(self.session_id, qs, "")
        schema = XMLSchema(schema_str)
        paths = schema.locate(CallID__ne='-1')
        logs = [schema.retrieve('__'.join(path.split('__')[:-1])) for path in paths]
        return sorted(logs, key=lambda x: int(x['CallID']), reverse=True)[0]
    
    @handle_response
    def get_physician(self, physician_id):
        args = [self.session_id, "PhysicianDetail", "Symed", f"@PhysicianID|{physician_id}|int"]
        return self.client.service.API_GetData(*args)

    def add_physician(self, office_id, **kwargs):
        args = [self.session_id, "Locations", "Provider", "PhysicianDetail_Create", 5, f"@OfficeID|{office_id}|int"]
        result = self.client.service.API_TreeDataCommand(*args)
        if "Error|" in result:
            raise APICallError(result)
        else:
            if len(kwargs) != 0:
                physician_id = result.split("|")[1]
                return self.edit_physician(physician_id, **kwargs)
            else:
                return result

    @handle_response
    def edit_physician(self, physician_id, **kwargs):
        physician = self.get_physician(physician_id)
        schema_and_data = ET.fromstring(physician)
        for key, value in kwargs.items():
            if schema_and_data[1][0].find(key) is None:
                new_attr = ET.SubElement(schema_and_data[1][0], key)
                new_attr.text = value
            else:
                schema_and_data[1][0].find(key).text = value
        updated_schema = ET.tostring(schema_and_data)
        args = [self.session_id, "Locations", "Provider", "PhysicianDetail",
                "Symed", f"@PhysicianID|{physician_id}|int", updated_schema]
        return self.client.service.API_UpdateData(*args)

    @handle_response
    def delete_physician(self, physician_id="", office_id=""):
        if not (physician_id and office_id):
            raise APICallError("You must specify both the physician_id and office_id.")
        args = [self.session_id, "Locations", "Provider", "PhysicianDetail_Delete", 6,
                f"@PhysicianID|{physician_id}|int@OfficeID|{office_id}|int"]
        return self.client.service.API_TreeDataCommand(*args)

    def get_latest_physician(self):
        qs = "SELECT * FROM PhysicianDetail"
        schema_str = self.client.service.API_GeneralQuery(self.session_id, qs, "")
        schema = XMLSchema(schema_str)
        paths = schema.locate(PhysicianID__ne='-1')
        physicians = [schema.retrieve('__'.join(path.split('__')[:-1])) for path in paths]
        return sorted(physicians, key=lambda x: int(x['PhysicianID']), reverse=True)[0]

    @handle_response
    def get_office(self, office_id):
        args = [self.session_id, "Office", "Symed", f"@OfficeID|{office_id}|int"]
        return self.client.service.API_GetData(*args)

    @handle_response
    def add_office(self, practice_id=""):
        if not practice_id:
            raise APICallError("You must provide a practice_id with which to associate this office.")
        args = [self.session_id, "Locations", "Office", "Offices_Create", 5, f"@PracticeID|{practice_id}|int"]
        return self.client.service.API_TreeDataCommand(*args)

    @handle_response
    def delete_office(self, office_id=""):
        if not office_id:
            raise APICallError("You must specify the office_id.")
        args = [self.session_id, "Locations", "Office", "Offices_Delete", 6,
                f"@OfficeID|{office_id}|int"]
        return self.client.service.API_TreeDataCommand(*args)

    def get_latest_office(self):
        qs = "SELECT * FROM Offices"
        schema_str = self.client.service.API_GeneralQuery(self.session_id, qs, "")
        schema = XMLSchema(schema_str)
        paths = schema.locate(OfficeID__ne='-1')
        offices = [schema.retrieve('__'.join(path.split('__')[:-1])) for path in paths]
        return sorted(offices, key=lambda x: int(x['OfficeID']), reverse=True)[0]

    def get_latest_practice(self):
        qs = "SELECT * FROM Offices WHERE OfficeID = PracticeID"
        schema_str = self.client.service.API_GeneralQuery(self.session_id, qs, "")
        schema = XMLSchema(schema_str)
        paths = schema.locate(OfficeID__ne='-1')
        offices = [schema.retrieve('__'.join(path.split('__')[:-1])) for path in paths]
        return sorted(offices, key=lambda x: int(x['OfficeID']), reverse=True)[0]
    
    @handle_response
    def get_contact_log(self, physician_id):
        guid = self._get_physician_guid(physician_id)
        print(guid)
        guid_string = '{' + guid + '}'
        guid = uuid.UUID(f'{guid_string}').hex
        print(guid)
        args = [self.session_id, "CallLog", "Symed", f'@EntityGuid|"{guid}"|Guid']
        args = [self.session_id, "CallLog", "Symed", f'@PhysicianID|{physician_id}|int']
        print(args)
        return self.client.service.API_GetData(*args)

    def __init__(self, settings=Settings()):
        self.endpoint = settings.ENDPOINT
        self.session = Session()
        self.client = Client(settings.WSDL_LOCATION, transport=Transport(session=self.session))
        if "Success" in self.client.service.API_Test():
            self.session_id = self.client.service.API_Login(settings.USERNAME, settings.PASSWORD).split("|")[1]
        else:
            raise APITestFailError("Test connection failed.")

In [369]:
connection = Connection()

In [370]:
# print(connection.get_latest_contact_log(12))

In [371]:
connection.get_contact_log(305)

1c71baf2-4edd-4391-8cd7-e40c0b38564d
1c71baf24edd43918cd7e40c0b38564d
['aaef28e7248e408c8df9e4af6cd079c5', 'CallLog', 'Symed', '@PhysicianID|305|int']


APICallError: Error|SymedData::LoadData - Procedure or function 'ContactLog_Select' expects parameter '@EntityGuid', which was not supplied.

In [309]:
print(connection._get_physician_guid(305))

1c71baf2-4edd-4391-8cd7-e40c0b38564d


Trying to get a CallLog entry from a Guid:

In [372]:
connection.client.service.API_GetData('aaef28e7248e408c8df9e4af6cd079c5', 
                                      'CallLog', 
                                      'Symed', 
                                      '@PhysicianID|305|int')

"Error|SymedData::LoadData - Procedure or function 'ContactLog_Select' expects parameter '@EntityGuid', which was not supplied."

In [304]:
# original UUID from PhysicianDetail
connection.client.service.API_GetData('8596a3ee624449988ff773841c874607', 
                                      'CallLog', 
                                      'Symed', 
                                      '@EntityGuid|1c71baf2-4edd-4391-8cd7-e40c0b38564d|Guid')

'Error|SymedData::LoadData - Failed to convert parameter value from a Int32 to a Guid.'

In [316]:
# hex representation of original UUID from PhysicianDetail
connection.client.service.API_GetData('98c6fe02496c4bd1a31d55bab681884c', 
                                      'CallLog', 
                                      'Symed', 
                                      '@EntityGuid|1c71baf24edd43918cd7e40c0b38564d|Guid')

'Error|SymedData::LoadData - Failed to convert parameter value from a Int32 to a Guid.'

In [315]:
# 128 bit representation of original UUID from PhysicianDetail
connection.client.service.API_GetData('98c6fe02496c4bd1a31d55bab681884c', 
                                      'CallLog', 
                                      'Symed', 
                                      '@EntityGuid|37808905152801010081679917987365672525|Guid')

'Error|SymedData::LoadData - Failed to convert parameter value from a Int32 to a Guid.'

In [346]:
# 32 bit representation of time_low portion of original UUID from PhysicianDetail
connection.client.service.API_GetData('577e64cf85d54e289c9cccb92253bf38', 
                                      'CallLog', 
                                      'Symed', 
                                      '@EntityGuid|477215474|Guid')

'Error|SymedData::LoadData - Failed to convert parameter value from a Int32 to a Guid.'

In [373]:
connection.client.service.API_GetData('8596a3ee624449988ff773841c874607', 
                                      'CallLog', 
                                      'Symed', 
                                      '@EntityGuid|"1c71baf2-4edd-4391-8cd7-e40c0b38564d"|Guid')

'Error|SymedData::LoadData - Failed to convert parameter value from a Int32 to a Guid.'

In [363]:
connection.client.service.API_GetData('d2e1dd8d9a08462d9853af93c2bbb57f', 
                                      'CallLog', 
                                      'Symed', 
                                      '@EntityGuid|"1c71baf24edd43918cd7e40c0b38564d"|Guid')

'Error|SymedData::LoadData - Failed to convert parameter value from a Int32 to a Guid.'